In [ ]:
   def lat_lng_to_grid(self,lat,lng):
        km_per_lat = 111
        km_per_lng = 111 * math.cos(math.radians((22.47+22.803) / 2))

        # 获取网格的原点
        origin_lng = self.shenzhen_range[0]  # 左下角的经度
        origin_lat = self.shenzhen_range[2]  # 左下角的纬度

        # 计算网格索引
        x = int((lng - origin_lng)*km_per_lng / self.size_per_grid)
        y = int((lat - origin_lat)*km_per_lat / self.size_per_grid)

    # 返回网格索引
        return x, y    
        
    def init_station(self,filename = './data/shenzhen_stations.csv'):
        
        stations = pd.read_csv(filename,dtype={'id': str})
        
        for index, row in stations.iterrows():
            stationid = row['id']
            x,y = self.lat_lng_to_grid(row['lat'],row['lng'])
            self.stations[stationid]=Station(stationid,self.truck_per_station,self.line_size,(x,y))
        
        
    def init_project(self,filename = './data/shenzhen_projects.csv'):
        
        projects = pd.read_csv(filename,dtype={'id': str})
        
        for index, row in projects.iterrows():
            id = row['id']
            x,y = self.lat_lng_to_grid(row['lat'],row['lng'])
            self.projects[id]=(x,y)
            
    def init_orders(self,filename = './data/shenzhen_orders_0324.csv'):
        fields_to_keep = ['project_id','create_time','deliver_time','order_quantity','ticket_count','pouring_type','order_id']
        type_conversions = {'project_id': str,'order_quantity': float,'ticket_count':int,'order_id':str}
        df = pd.read_csv(filename, usecols=fields_to_keep, dtype=type_conversions,parse_dates=['create_time','deliver_time'])
        
        # 将 Timestamp 转换为 datetime
        # 将 deliver_time 列转换为 datetime 类型
        df['deliver_time'] = pd.to_datetime(df['deliver_time'])
        df['create_time'] = pd.to_datetime(df['create_time'])
        
        # 不妨假设deliver_time是真实的送货时间，以此进行筛选 
        filtered_df = df[(df['deliver_time'] >= self.start_date) & (df['deliver_time'] <= self.end_date)]
        
        self.orders = filtered_df.to_dict(orient='records')
        for index, order in enumerate(self.orders, start=1):
            order['id'] = index
        
    def distance_between_grids(self,x1,y1,x2,y2):
        return (abs(x1-x2)+abs(y1-y2))*self.size_per_grid    
    
    
    
    def init_interaction(self):
        import csv
        project_station_interaction = {}
        # 读取深圳的订单 CSV 文件
        with open('./data/shenzhen_orders_0324.csv','r',encoding='utf-8') as f:
            reader = csv.DictReader(f)
            
            for row in reader:
                project_id=row['project_id']
                station_id=row['station_id']
                
                if project_id not in project_station_interaction:
                    project_station_interaction[project_id]={}
                    
                if station_id not in project_station_interaction[project_id]:

                    project_loc = self.projects[project_id]
                    station_loc = self.stations[station_id].coord
                    
                    project_station_interaction[project_id][station_id]=self.distance_between_grids(project_loc[0],project_loc[1],station_loc[0],station_loc[1])
                
        self.interaction = project_station_interaction
        
        
    def init_interaction2(self):
        
        project_station_interaction = {}
        
        for project_id, project_loc in self.projects.items():
            project_station_interaction[project_id] = {}
            
            for station_id, station in self.stations.items():
                station_loc = station.coord
                distance = self.distance_between_grids(project_loc[0], project_loc[1], station_loc[0], station_loc[1])
                
                if distance >30:
                    continue
                # 只记录距离小于30km的厂站
                project_station_interaction[project_id][station_id] = distance
                
        self.interaction = project_station_interaction